In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [ ]:
movies.head(1)

In [ ]:
movies = movies.merge(credits, on='title')
movies.head(1)

In [ ]:
# genres
# id
# keywords
# title
# overview
# cast
# crew

movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
movies.head(1)

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)
len(movies)

In [ ]:
movies.duplicated().sum()


In [ ]:
movies.drop_duplicates(subset='title', keep='first', inplace=True)
len(movies)

In [ ]:
movies.iloc[0].genres

In [ ]:
# Convert the above to ['Action', 'Adventure', 'Fantasy', 'Science Fiction']
import ast
def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L


In [ ]:
movies['genres'] = movies['genres'].apply(convert)

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)

In [ ]:
def convert3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter += 1
        else:
            break
    return L

In [ ]:
movies['cast'] = movies['cast'].apply(convert3)
movies.head()

In [ ]:
movies['crew'][0]

In [ ]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L

In [ ]:
movies['crew'] = movies['crew'].apply(fetch_director)
movies.head()

In [ ]:
# converting overview to list of string
# we have converted it to a list so that we can concatenate all the columns to form a single column
movies['overview'] = movies['overview'].apply(lambda x: x.split())
movies.head()

In [ ]:
# remove spaces between the words
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(' ', '') for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(' ', '') for i in x])
movies.head()

In [ ]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']
new_df = movies[['movie_id', 'title', 'tags']]
new_df.head()


In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))
new_df.head()

In [ ]:
new_df['tags'][0]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())
new_df.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [ ]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
!pip install nltk
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    
    return " ".join(y)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stem)


In [ ]:
new_df.head()
np.set_printoptions(threshold=np.inf)

In [ ]:
cv.get_feature_names_out()   ## list of all the words in the vectors

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vectors)

In [ ]:
similarity

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('Batman Begins')

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity, open('similarity.pkl', 'wb'))